# Quantum Teleportation

$\providecommand{\ket}[1]{\left|#1\right\rangle}$
$\providecommand{\bra}[1]{\left\langle#1\right|}$

In this sample, we will be looking at using quantum teleportation on the Azure Quantum service.

To refresh our mind let us review how quantum teleportation works.

We have two parties, Alice and Bob, they would like to share information, and they have an entangled state (usually the Bell state $\frac{1}{\sqrt{2}} (\ket{00} + \ket{11})$). 
If Alice now wishes to transfer quantum information, she will entangle some payload with her qubit and measure her qubit and payload.She will transmit the results of the measurement to Bob, who can use them to reconstruct Alice's payload on his entangled qubit.

In the following code we will implement this operation.

Before that, let's connect to the Az Quantum service and set our ta

In [1]:
operation PrepareBellState(alice : Qubit, bob : Qubit) : Unit is Adj + Ctl {
    H(alice);
    CNOT(alice, bob);
}

operation Send(alice : Qubit, payload: Qubit) : (Result, Result) {
    CNOT(payload, alice);
    H(payload);
    return (M(alice), M(payload));
}

operation Receive(bob : Qubit, (cr_x : Result, cr_z : Result)) : Unit {
    if cr_x == One { X(bob); }
    if cr_z == One { Z(bob); }
}

PrepareBellState, Send, Receive

We will now use Q# visualization tools to show the unitary responsible from creating the Bell state.
Printing unitary transformations implemented by operations is a great way to check whether your implementation is mathematically correct (for small operations).

In [2]:
open Microsoft.Quantum.Diagnostics; // Contains DumpOperation

operation DumpBellState() : Unit {
    DumpOperation(2, qubits => PrepareBellState(qubits[0], qubits[1]));
}

DumpBellState, _abfda0ce01a6474fb8dba0efaab253cb_DumpBellState

In [3]:
%simulate DumpBellState

Real:
[[0.7071067811865477, 0.7071067811865477, 0, 0], 
[0, 0, 0.7071067811865477, -0.7071067811865477], 
[0, 0, 0.7071067811865477, 0.7071067811865477], 
[0.7071067811865477, -0.7071067811865477, 0, 0]]
Imag:
[[0, 0, 0, 0], 
[0, 0, 0, 0], 
[0, 0, 0, 0], 
[0, 0, 0, 0]]

()

Below we will now verify that our teleportation does indeed work correctly.

In order to do this, we will teleport a state of a qubit that we rotate to an unusual angle from Alice to Bob.

We will be using the `DumpRegister` and `AssertQubit` functions from the `Microsoft.Quantum.Diagnostics` to introspect our registers and assert that by the end of the computation the state of Bob's qubit is equivalent to the state of the payload qubit before teleportation.

In [7]:
operation PerformTeleportationOnBellState(alice : Qubit, bob : Qubit, payload : Qubit) : Unit {
    let cr = Send(alice, payload);
    Receive(bob, cr);
}

PerformTeleportationOnBellState

In [4]:
operation TestTeleportation() : Unit {
    use (alice, bob, payload) = (Qubit(), Qubit(), Qubit());
    Ry(1.727, payload);
    DumpRegister((), [payload]);
    PrepareBellState(alice, bob);
    PerformTeleportationOnBellState(alice, bob, payload);
    Adjoint Ry(1.727, bob); // Uncompute Bob's qubi, returning it to the |0> state
    AssertQubit(Zero, bob);
}

Error QS5022: No identifier with the name "PerformTeleportationOnBellState" exists.
 5 |     PrepareBellState(alice, bob);
 6 |     PerformTeleportationOnBellState(alice, bob, payload);
   |     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 7 |     Adjoint Ry(1.727, bob); // Uncompute Bob's qubi, returning it to the |0> state

In [ ]:
%simulate TestTeleportation

Seeing empirically that our teleportation works, we can proceed to run it on hardware.

For this we will recreate our test function but run it multiple times.

In [ ]:
open Microsoft.Quantum.Convert;

In [ ]:
operation TeleportToBob() : Result {
    use (alice, bob, payload) = (Qubit(), Qubit(), Qubit());
    Ry(1.727, payload);
    PrepareBellState(alice, bob);
    PerformTeleportationOnBellState(alice, bob, payload);
    Adjoint Ry(1.727, bob);
    return M(bob);
}

operation CountCorrectTeleportation(n : Int) : Double {
    let expected = Count(res -> res == Zero, DrawMany(TeleportToBob, n))
    return IntAsDouble(expected) / IntAsDouble(n);
}

Let us now see if our implementation is correct by seeing if the simulator succeeds in the teleportation 100% of the time.

In [ ]:
%simulate CountCorrectTeleportation n=100

We see that on a perfect machine the teleportation works correctly every time. Now let us try running our work on Azure Quantum.

When running jobs on Azure Quantum, before going to real hardware it is recommended to run your code through a validator to make sure you don't waste precious resources on faulty code. `quantinuum.hqs-lt-s2-apival` is the appropriate target for this.

In [ ]:
%azure.connect resourceId="" location=""

In [ ]:
%azure.target quantinuum.hqs-lt-s2-apival

Let us now submit our job

In [ ]:
%azure.submit TeleportToBob jobName="Teleportation API validaton"

Now with the confidence that our job will work we can submit it to the simulator. The Quantinuum emulator provided by Azure Quantum differs from the simulator running locally, in that it accurately models noise and behavior of the Quantinuum quantum computer.

In [ ]:
%azure.target quantinuum.hqs-lt-s2-sim

In [ ]:
%azure.submit TeleportToBob jobName="Teleporation simulation"

Now we will look into our job and wait for its completion using `%azure.status` and once complete see results using `%azure.output`.

In [ ]:
%azure.status

In [ ]:
%azure.output

Let us now enhance our code by building entanglement swapping. The idea of entanglement swapping is that if Alice and Bob do not have a direct connection but connections to third parties such that a path exists, then each pair of adjacent parties can create Bell states and teleport Alice's second qubit along this path until it reaches Bob.

First we build the base case of entanglement swapping `EntanglementSwap3`, then we proceed to build the general case `EntanglementSwapN`

We will test entanglement swapping with 3 parties by using the Bell pair we create to send a specific qubit state. With the advanced debugging tools of Q# we can introspect that states mid-simulation allowing us to perform this test. To validate that teleportation protocol was implemented correctly, we will use assertion features provided by Q#. Later in this notebook we will test entanglement swapping by checking that Alice and Bob have a Bell state since we will run on real hardware (or accurate simulators thereof) which does not allow such introspection. Validating concepts on small scale using Q# debugging features before proceeding to building full-scale algorithms is a good quantum software development practice.

In [8]:
operation EntanglementSwap3(aliceCharlie : Qubit[], charlieBob : Qubit[]) : (Qubit, Qubit) {
    PrepareBellState(aliceCharlie[0], aliceCharlie[1]);
    PrepareBellState(charlieBob[0], charlieBob[1]);
    PerformTeleportationOnBellState(charlieBob[0], charlieBob[1], aliceCharlie[1]);
    ResetAll([aliceCharlie[1], charlieBob[0]]);
    return (aliceCharlie[0], charlieBob[1]);
}

operation TestEntanglementSwap3() : Unit {
    use aliceCharlie = Qubit[2];
    use charlieBob = Qubit[2];
    let (alice, bob) = EntanglementSwap3(aliceCharlie, charlieBob);
    use payload = Qubit();
    Ry(1.727, payload);
    DumpRegister((), [payload]);
    PerformTeleportationOnBellState(alice, bob, payload);
    Adjoint Ry(1.727, bob);
    AssertQubit(Zero, bob);
}

EntanglementSwap3, TestEntanglementSwap3

In [9]:
%simulate TestEntanglementSwap3

|0⟩	0.6497810284339186 + 0𝑖
|1⟩	0.7601214475906853 + 0𝑖

()

Now let us build entanglement swapping for $n$ parties, given that we have validated the concept for 3 parties. As mentioned above, we will then test that Alice and Bob end up sharing a Bell state.

In [10]:
operation EntanglementSwapN(nParties : Int, qubits : Qubit[]) : (Qubit, Qubit) {
    for i in 0..nParties-1 {
        // Entangle pairs of qubits
        PrepareBellState(qubits[2 * i], qubits[2 * i + 1]);
    }
    for i in 1..nParties-1 {
        // Teleport previously teleported qubit 
        PerformTeleportationOnBellState(qubits[2 * i], qubits[2 * i + 1], qubits[2 * i - 1]);
    }
    return (qubits[0], qubits[2 * nParties - 1]);
}

EntanglementSwapN

Let us now test entanglement swapping with 4 qubits locally by teleporting a state again and using Q#'s debugging features.

In [11]:
operation EntanglementSwap4() : (Result) {
    use qubits = Qubit[2 * 4];
    let (q1, qn) = EntanglementSwapN(4, qubits);
    use payload = Qubit();
    let rot = Ry(1.727, _);
    rot(payload);
    DumpRegister((), [payload]);
    PerformTeleportationOnBellState(q1, qn, payload);
    DumpRegister((), [qn]);
    Adjoint rot(qn);
    return M(qn);
}

EntanglementSwap4

In [12]:
%simulate EntanglementSwap4

|0⟩	0.6497810284339186 + 0𝑖
|1⟩	0.7601214475906852 + 0𝑖

|0⟩	0.6497810284339188 + 0𝑖
|1⟩	0.7601214475906855 + 0𝑖

Zero

In [17]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;

operation TeleportNParties(nParties : Int) : Result {
    use qubits = Qubit[2 * nParties];
    let (alice, bob) = EntanglementSwapN(nParties, qubits);
    use payload = Qubit();
    let rot = Ry(1.727, _);
    rot(payload);
    PerformTeleportationOnBellState(alice, bob, payload);
    Adjoint rot(bob);
    return M(bob);
}

operation CountCorrectTeleportationNParties(n : Int, nParties : Int) : Double {
    let success = Count(res -> res == Zero, DrawMany(TeleportNParties, n, nParties));
    return IntAsDouble(success) / IntAsDouble(n);
}

TeleportNParties, CountCorrectTeleportationNParties, _1601ceffef5648579c6768edf78b3ea2_CountCorrectTeleportationNParties

Given that we will run on Quantinuum's H1-2, we can run with up to 6 parties as we have 12 qubits available. So let us run one example job.

**Please note that this sample makes use of paid services on Azure Quantum. The cost of running this sample with the provided parameters on Quantinuum in a free trial subscription is approximately 31.8EHQC. This quantity is only an approximate estimate and should not be used as a binding reference. The cost of the service might vary depending on your region, demand and other factors.** 


In [19]:
%simulate CountCorrectTeleportationNParties n=100 nParties=6

1

In [ ]:
%azure.target quantinuum.hqs-lt-s2-apival

In [ ]:
%azure.submit TeleportNParties nParties=5 jobName="Entanglement Swapping - 6 parties API validation"

In [ ]:
%azure.status

In [ ]:
%azure.output

In [ ]:
%azure.target quantinuum.hqs-lt-s2-sim

In [ ]:
%azure.submit TeleportNParties nParties=5 jobName="Entanglement Swapping - 6 parties"

In [ ]:
%azure.status

In [ ]:
%azure.output

As we can see we were able to run entanglement swapping on the Quantinuum emulator and were able to use the mid-circuit measurement capability in the process. We also saw how we could use the Azure Quantum service to submit and process our jobs.